In [9]:
#Let's mount the google drive with colab:

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.preprocessing import LabelEncoder

# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings("ignore")

In [52]:
#Loading Loan Accepted Dataset from Google Drive:

dataset = pd.read_csv("/content/drive/MyDrive/CSVs/Clean Credit Card/clean_data.csv") #Path of my Google drive source
dataset.head()

,ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
0,5008804,1,1,1,1,0,0,0,0,2,15,427500.0,32.868574,12.435574,Working,Higher education,Civil marriage,Rented apartment,Other,1
1,5008806,1,1,1,0,0,0,0,0,2,29,112500.0,58.793815,3.104787,Working,Secondary / secondary special,Married,House / apartment,Security staff,0
2,5008808,0,0,1,0,1,1,0,0,1,4,270000.0,52.321403,8.353354,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,0
3,5008812,0,0,1,0,0,0,1,0,1,20,283500.0,61.504343,0.000000,Pensioner,Higher education,Separated,House / apartment,Other,0
4,5008815,1,1,1,1,1,1,0,0,2,5,270000.0,46.193967,2.105450,Working,Higher education,Married,House / apartment,Accountants,0


In [53]:
#Encoding for the categorical values
#Make it easier for the model to work

categorical_columns = ['Income_type', 'Education_type', 'Family_status', 'Occupation_type', 'Housing_type']
label = LabelEncoder()

for col in categorical_columns:
  dataset[col] = label.fit_transform(dataset[col])

In [54]:
from google.colab import files
dataset.to_csv("Final_Dataset.csv")
files.download("Final_Dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
print("This WBCD dataset is consisted of",dataset.shape)

This WBCD dataset is consisted of (9709, 20)


In [30]:
# Divide "WBCD data" into Train(70%) / Test data(30%)
train, test = train_test_split(dataset, test_size=0.3, random_state=42)
print("Training Data :",  train.shape)
print("Testing Data :", test.shape)

Training Data : (6796, 20)
Testing Data : (2913, 20)


**Concerning the ID:**

Save the ID column for later combination(results).

Drop the ID column in train, test datasets, because it's unnecessary for model learning predictions.

In [31]:
# Save the ID column for later combination(results).

train_id = train['ID']
test_id = test['ID']

# Drop the ID column in train, test datasets, because it's unnecessary for model learning predictions.

train_data = train.iloc[:,1:]
test_data = test.iloc[:,1:]

print("Training Data :",train_data.shape)
print("Testing Data :",test_data.shape)

Training Data : (6796, 19)
Testing Data : (2913, 19)


In [32]:
train_data.columns

Index(['Gender', 'Own_car', 'Own_property', 'Work_phone', 'Phone', 'Email',
       'Unemployed', 'Num_children', 'Num_family', 'Account_length',
       'Total_income', 'Age', 'Years_employed', 'Income_type',
       'Education_type', 'Family_status', 'Housing_type', 'Occupation_type',
       'Target'],
      dtype='object')

**Seperate by x_data, y_data:**

x_data : columns(features to predict diagnosis) for training. (eliminate target)
y_data : columns for comparing with predictions results. (need original target)

In [33]:
#Training Data
train_x = train_data.loc[:, train_data.columns != 'Target']
print("Training Data :", train_x.shape)

# Testing Data
test_x = test_data.loc[:, train_data.columns != 'Target']
print("Testing Data :", test_x.shape)

Training Data : (6796, 18)
Testing Data : (2913, 18)


In [34]:
# Training Data
train_y = train_data.loc[:, train_data.columns == 'Target']
print("Training Data :", train_y.shape)

# Testing Data
test_y = test_data.loc[:, train_data.columns == 'Target']
print("Testing Data :", test_y.shape)

Training Data : (6796, 1)
Testing Data : (2913, 1)


**Making ANN-SLP Model**

Placeholder is one of the function in tensorflow. It is a space to put and change values while the program is running.

for X, a place must have 18 columns, since dataset data has 18 features.
for Y, a place must have 1 columns, since the results has 1 outcome.
If you see the row "None", it means it has no size limits. (You can write -1 instead of "None")

In [35]:
X = tf.placeholder(tf.float32, [None, 18])
Y = tf.placeholder(tf.float32, [None, 1])

**Making the weights and the bias**

In [38]:
# weight
W = tf.Variable(tf.random_normal([18,1], seed=0), name='weight')

# bias
b = tf.Variable(tf.random_normal([1], seed=0), name='bias')

**Making Output Results**
Output = Weight * Input + Bias
tf.matmul() : for array multiply

In [39]:
logits = tf.matmul(X,W) + b

In [40]:
hypothesis = tf.nn.sigmoid(logits)

cost_i = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=Y)
cost = tf.reduce_mean(cost_i)
# cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))

In [41]:
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [42]:
prediction = tf.cast(hypothesis > 0.5, dtype=tf.float32)
correct_prediction = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

In [43]:
dataset.head()

,ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
0,5008804,1,1,1,1,0,0,0,0,2,15,427500.0,32.868574,12.435574,4,1,0,4,12,1
1,5008806,1,1,1,0,0,0,0,0,2,29,112500.0,58.793815,3.104787,4,4,1,1,17,0
2,5008808,0,0,1,0,1,1,0,0,1,4,270000.0,52.321403,8.353354,0,4,3,1,15,0
3,5008812,0,0,1,0,0,0,1,0,1,20,283500.0,61.504343,0.000000,1,1,2,1,12,0
4,5008815,1,1,1,1,1,1,0,0,2,5,270000.0,46.193967,2.105450,4,1,1,1,0,0


In [44]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10001):
        sess.run(train, feed_dict={X: train_x, Y: train_y})
        if step % 1000 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X: train_x, Y: train_y})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))
            
    train_acc = sess.run(accuracy, feed_dict={X: train_x, Y: train_y})
    test_acc, test_predict, test_correct = sess.run([accuracy,prediction,correct_prediction], feed_dict={X: test_x, Y: test_y})
    print("Model Prediction =", train_acc)
    print("Test Prediction =", test_acc)

Step:     0	Loss: 379232480.000	Acc: 87.21%
Step:  1000	Loss: 133321320.000	Acc: 87.21%
Step:  2000	Loss: 324600384.000	Acc: 87.21%
Step:  3000	Loss: 78689296.000	Acc: 87.21%
Step:  4000	Loss: 269968384.000	Acc: 87.21%
Step:  5000	Loss: 24057256.000	Acc: 87.21%
Step:  6000	Loss: 215336336.000	Acc: 87.21%
Step:  7000	Loss: 200068176.000	Acc: 12.79%
Step:  8000	Loss: 160704304.000	Acc: 87.21%
Step:  9000	Loss: 351983360.000	Acc: 87.21%
Step: 10000	Loss: 106072288.000	Acc: 87.21%
Model Prediction = 0.8721307
Test Prediction = 0.85787845


In [48]:
def ann_slp():
    print("===========Data Summary===========")
    print("Training Data :", train_x.shape)
    print("Testing Data :", test_x.shape)

    X = tf.placeholder(tf.float32, [None,18])
    Y = tf.placeholder(tf.float32, [None, 1])

    W = tf.Variable(tf.random_normal([18,1], seed=0), name='weight')
    b = tf.Variable(tf.random_normal([1], seed=0), name='bias')

    logits = tf.matmul(X,W) + b
    hypothesis = tf.nn.sigmoid(logits)
    
    cost_i = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=Y)
    cost = tf.reduce_mean(cost_i)

    train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

    prediction = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    correct_prediction = tf.equal(prediction, Y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

    print("\n============Processing============")
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for step in range(10001):
            sess.run(train, feed_dict={X: train_x, Y: train_y})
            if step % 1000 == 0:
                loss, acc = sess.run([cost, accuracy], feed_dict={X: train_x, Y: train_y})
                print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))

        train_acc = sess.run(accuracy, feed_dict={X: train_x, Y: train_y})
        test_acc,test_predict,test_correct = sess.run([accuracy,prediction,correct_prediction], feed_dict={X: test_x, Y: test_y})
        
        print("\n============Results============")
        print("Model Prediction =", train_acc)
        print("Test Prediction =", test_acc)
        
        return train_acc,test_acc
    
ann_slp_train_acc, ann_slp_test_acc = ann_slp()

===========Data Summary===========
Training Data : (6796, 18)
Testing Data : (2913, 18)

============Processing============
Step:     0	Loss: 379232480.000	Acc: 87.21%
Step:  1000	Loss: 133321320.000	Acc: 87.21%
Step:  2000	Loss: 324600384.000	Acc: 87.21%
Step:  3000	Loss: 78689296.000	Acc: 87.21%
Step:  4000	Loss: 269968384.000	Acc: 87.21%
Step:  5000	Loss: 24057256.000	Acc: 87.21%
Step:  6000	Loss: 215336336.000	Acc: 87.21%
Step:  7000	Loss: 200068176.000	Acc: 12.79%
Step:  8000	Loss: 160704304.000	Acc: 87.21%
Step:  9000	Loss: 351983360.000	Acc: 87.21%
Step: 10000	Loss: 106072288.000	Acc: 87.21%

============Results============
Model Prediction = 0.8721307
Test Prediction = 0.85787845


In [49]:
sub = pd.DataFrame()
sub['ID'] = test_id
sub['Predict_Type'] = test_predict.astype(int)
sub['Origin_Type'] = test_y
sub['Correct'] = test_correct
sub.head(10)

,ID,Predict_Type,Origin_Type,Correct
7591,5115836,0,1,False
6760,5100394,0,0,True
8782,5137353,0,0,True
4656,5068622,0,0,True
1915,5033758,0,1,False
7499,5114598,0,0,True
8607,5135591,0,0,True
5834,5091280,0,0,True
1224,5025123,0,0,True
3272,5050647,0,0,True


In [50]:
dataset.head()

,ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
0,5008804,1,1,1,1,0,0,0,0,2,15,427500.0,32.868574,12.435574,4,1,0,4,12,1
1,5008806,1,1,1,0,0,0,0,0,2,29,112500.0,58.793815,3.104787,4,4,1,1,17,0
2,5008808,0,0,1,0,1,1,0,0,1,4,270000.0,52.321403,8.353354,0,4,3,1,15,0
3,5008812,0,0,1,0,0,0,1,0,1,20,283500.0,61.504343,0.000000,1,1,2,1,12,0
4,5008815,1,1,1,1,1,1,0,0,2,5,270000.0,46.193967,2.105450,4,1,1,1,0,0


In [51]:
sub[['ID','Predict_Type']].to_csv('submission.csv',index=False)